<a href="https://colab.research.google.com/github/gnoejh/ict1022/blob/main/Architectures/dbn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Belief Networks (DBN)

Deep Belief Networks are probabilistic generative models composed of multiple layers of stochastic, latent variables. They represent one of the earliest successful deep learning architectures and pioneered many of the techniques used in modern deep learning.

## Historical Context

Deep Belief Networks were introduced by Geoffrey Hinton and colleagues in 2006, during what is now called the "Deep Learning Renaissance." Prior to this work:

1. Training deep neural networks was extremely difficult due to the vanishing gradient problem
2. There was no effective way to pre-train deep architectures
3. Unsupervised learning in deep networks was largely unexplored

Hinton's breakthrough came with the realization that deep networks could be trained effectively by:

1. Pre-training each layer as a Restricted Boltzmann Machine (RBM)
2. Stacking these RBMs to form a deep architecture
3. Fine-tuning the entire network using supervised learning

This work helped revitalize neural network research and laid the groundwork for the deep learning revolution.

## Architecture

A Deep Belief Network consists of:

1. **Multiple Layers** of Restricted Boltzmann Machines (RBMs)
2. **Visible Layer** that receives input data
3. **Multiple Hidden Layers** that capture increasingly abstract features

![DBN Architecture](https://miro.medium.com/max/700/1*WGBv7xMNL-StLmfX7qoGZA.png)

The key insight behind DBNs is that each layer can be trained greedily, one at a time, to learn a representation of its input. The trained layers are then stacked to form a deep network.

## Restricted Boltzmann Machines (RBMs)

Before diving deeper into DBNs, we need to understand the building blocks: Restricted Boltzmann Machines.

An RBM is a two-layer, undirected graphical model consisting of:
- A visible (input) layer
- A hidden layer
- Connections between the visible and hidden layers (but no connections within each layer)

The energy function of an RBM is defined as:

$$E(\mathbf{v}, \mathbf{h}) = -\sum_i a_i v_i - \sum_j b_j h_j - \sum_{i,j} v_i w_{ij} h_j$$

Where:
- $\mathbf{v}$ is the vector of visible units
- $\mathbf{h}$ is the vector of hidden units
- $a_i$ and $b_j$ are the biases
- $w_{ij}$ are the weights connecting visible and hidden units

From this energy function, we can derive the joint probability distribution:

$$P(\mathbf{v}, \mathbf{h}) = \frac{1}{Z} e^{-E(\mathbf{v}, \mathbf{h})}$$

Where $Z$ is the partition function (normalizing constant).

## Training RBMs with Contrastive Divergence

RBMs are trained using an algorithm called Contrastive Divergence (CD), which approximates the gradient of the log-likelihood. The steps are:

1. Start with a training example $\mathbf{v}^{(0)}$ (data phase)
2. Sample hidden units $\mathbf{h}^{(0)}$ given $\mathbf{v}^{(0)}$
3. Sample visible units $\mathbf{v}^{(1)}$ given $\mathbf{h}^{(0)}$ (reconstruction phase)
4. Sample hidden units $\mathbf{h}^{(1)}$ given $\mathbf{v}^{(1)}$
5. Update weights: $\Delta w_{ij} = \epsilon (v_i^{(0)} h_j^{(0)} - v_i^{(1)} h_j^{(1)})$

In practice, CD-k is often used, where k steps of alternating Gibbs sampling are performed.

## RBM Implementation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class RBM:
    def __init__(self, num_visible, num_hidden, learning_rate=0.1):
        # Initialize weights and biases
        self.W = np.random.normal(0, 0.1, (num_visible, num_hidden))
        self.visible_bias = np.zeros(num_visible)
        self.hidden_bias = np.zeros(num_hidden)
        self.learning_rate = learning_rate
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sample_hidden(self, visible_prob):
        # Calculate hidden activation
        hidden_activation = np.dot(visible_prob, self.W) + self.hidden_bias
        hidden_prob = self.sigmoid(hidden_activation)
        hidden_states = np.random.binomial(1, hidden_prob)
        return hidden_prob, hidden_states
    
    def sample_visible(self, hidden_prob):
        # Calculate visible activation
        visible_activation = np.dot(hidden_prob, self.W.T) + self.visible_bias
        visible_prob = self.sigmoid(visible_activation)
        visible_states = np.random.binomial(1, visible_prob)
        return visible_prob, visible_states
    
    def contrastive_divergence(self, data, k=1):
        # Positive phase
        pos_hidden_prob, pos_hidden_states = self.sample_hidden(data)
        
        # Initialize chain for negative phase
        hidden_states = pos_hidden_states
        
        # Gibbs sampling
        for _ in range(k):
            vis_prob, _ = self.sample_visible(hidden_states)
            hidden_prob, hidden_states = self.sample_hidden(vis_prob)
        
        # Compute gradients and update weights
        pos_associations = np.dot(data.T, pos_hidden_prob)
        neg_associations = np.dot(vis_prob.T, hidden_prob)
        
        self.W += self.learning_rate * (pos_associations - neg_associations) / data.shape[0]
        self.visible_bias += self.learning_rate * np.mean(data - vis_prob, axis=0)
        self.hidden_bias += self.learning_rate * np.mean(pos_hidden_prob - hidden_prob, axis=0)
        
        # Compute reconstruction error
        error = np.mean(np.sum((data - vis_prob) ** 2, axis=1))
        return error
    
    def train(self, data, epochs=100, batch_size=10, k=1):
        num_samples = data.shape[0]
        num_batches = num_samples // batch_size
        
        errors = []
        
        for epoch in range(epochs):
            epoch_error = 0
            np.random.shuffle(data)
            
            for batch in range(num_batches):
                start = batch * batch_size
                end = start + batch_size
                batch_data = data[start:end]
                
                batch_error = self.contrastive_divergence(batch_data, k)
                epoch_error += batch_error
            
            avg_error = epoch_error / num_batches
            errors.append(avg_error)
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Reconstruction Error: {avg_error:.4f}")
        
        return errors
    
    def reconstruct(self, data):
        hidden_prob, _ = self.sample_hidden(data)
        visible_prob, _ = self.sample_visible(hidden_prob)
        return visible_prob

## Deep Belief Network Implementation

In [ ]:
class DBN:
    def __init__(self, layer_sizes, learning_rate=0.1):
        """Initialize a Deep Belief Network.
        
        Args:
            layer_sizes: List of layer sizes, from input to output
            learning_rate: Learning rate for RBMs
        """
        self.rbm_layers = []
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        
        # Create RBM layers
        for i in range(len(layer_sizes) - 1):
            self.rbm_layers.append(RBM(layer_sizes[i], layer_sizes[i+1], learning_rate))
    
    def pretrain(self, data, epochs=10, batch_size=10, k=1):
        """Greedy layer-wise pretraining of DBN."""
        input_data = data
        
        for i, rbm in enumerate(self.rbm_layers):
            print(f"\nTraining RBM layer {i+1}/{len(self.rbm_layers)}")
            rbm.train(input_data, epochs=epochs, batch_size=batch_size, k=k)
            
            # Transform data for next layer
            hidden_probs = []
            for batch_start in range(0, len(input_data), batch_size):
                batch_end = min(batch_start + batch_size, len(input_data))
                batch_data = input_data[batch_start:batch_end]
                hidden_prob, _ = rbm.sample_hidden(batch_data)
                hidden_probs.append(hidden_prob)
            
            input_data = np.vstack(hidden_probs)
    
    def generate(self, num_samples=1, k=1000):
        """Generate samples from the DBN."""
        # Initialize with random values in the top layer
        top_layer_size = self.layer_sizes[-1]
        samples = np.random.binomial(1, 0.5, size=(num_samples, top_layer_size))
        
        # Generate from the top-level RBM with Gibbs sampling
        top_rbm = self.rbm_layers[-1]
        for _ in range(k):
            visible_prob, _ = top_rbm.sample_visible(samples)
            hidden_prob, samples = top_rbm.sample_hidden(visible_prob)
        
        # Propagate downwards through the network
        for i in range(len(self.rbm_layers) - 2, -1, -1):
            rbm = self.rbm_layers[i]
            visible_prob, samples = rbm.sample_visible(samples)
        
        return visible_prob
    
    def reconstruct(self, data):
        """Reconstruct data through the DBN."""
        # Forward pass
        h = data
        for rbm in self.rbm_layers:
            hidden_prob, _ = rbm.sample_hidden(h)
            h = hidden_prob
        
        # Backward pass
        for i in range(len(self.rbm_layers) - 1, -1, -1):
            rbm = self.rbm_layers[i]
            visible_prob, _ = rbm.sample_visible(h)
            h = visible_prob
        
        return h

## Example: MNIST Digit Recognition

Let's train a DBN on the MNIST dataset and visualize the learned features.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

# Load MNIST dataset
try:
    X, y = fetch_openml("mnist_784", version=1, return_X_y=True)
except:
    # Fallback if OpenML is not available
    from sklearn.datasets import load_digits
    digits = load_digits()
    X = digits.data / 16.0  # Scale to [0,1]
    y = digits.target
    print("Using sklearn digits dataset (8x8 images) as fallback")
else:
    # Scale MNIST to [0,1]
    X = X / 255.0
    print("Loaded MNIST dataset successfully")

# Take a subset for faster training
n_samples = 5000
X_subset = X[:n_samples]

# Create and train DBN
input_size = X_subset.shape[1]  # 784 for MNIST, 64 for digits
dbn = DBN(layer_sizes=[input_size, 250, 100], learning_rate=0.01)

# Train with small number of epochs for demonstration
dbn.pretrain(X_subset, epochs=5, batch_size=10, k=1)

## Visualize Learned Features

Let's visualize what the first layer of the DBN has learned:

In [ ]:
def plot_weights(weights, img_shape, grid_size):
    """Plot weights as images in a grid."""
    fig, axes = plt.subplots(grid_size[0], grid_size[1], figsize=(12, 12))
    
    # Reshape and scale weights for visualization
    weights = weights.reshape([-1] + list(img_shape))
    
    for i, ax in enumerate(axes.flat):
        if i < len(weights):
            # Plot weights
            ax.imshow(weights[i], cmap='gray', interpolation='nearest')
            ax.set_xticks([])
            ax.set_yticks([])
        else:
            ax.axis('off')
    
    plt.suptitle("RBM Hidden Unit Filters", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# Get the weights from the first RBM layer
weights = dbn.rbm_layers[0].W

# Determine image shape (28x28 for MNIST, 8x8 for digits)
if input_size == 784:
    img_shape = (28, 28)
else:  # For the digits dataset
    img_shape = (8, 8)

# Plot 25 hidden units (5x5 grid)
plot_weights(weights.T[:25], img_shape, (5, 5))

## Reconstruct Images

Let's see how well our DBN can reconstruct some input images:

In [ ]:
# Select test images
test_images = X[:10]  # First 10 images

# Reconstruct images
reconstructed = dbn.reconstruct(test_images)

# Plot original and reconstructed images
fig, axes = plt.subplots(2, 10, figsize=(15, 4))

for i in range(10):
    # Original image
    if input_size == 784:
        axes[0, i].imshow(test_images[i].reshape(28, 28), cmap='gray')
        axes[1, i].imshow(reconstructed[i].reshape(28, 28), cmap='gray')
    else:  # For digits dataset
        axes[0, i].imshow(test_images[i].reshape(8, 8), cmap='gray')
        axes[1, i].imshow(reconstructed[i].reshape(8, 8), cmap='gray')
    
    axes[0, i].set_xticks([])
    axes[0, i].set_yticks([])
    axes[1, i].set_xticks([])
    axes[1, i].set_yticks([])

axes[0, 0].set_ylabel('Original')
axes[1, 0].set_ylabel('Reconstructed')

plt.tight_layout()
plt.show()

## Fine-tuning DBNs for Classification

After pre-training, a DBN can be fine-tuned by adding a classification layer and training with backpropagation. This approach was common before the widespread adoption of end-to-end training methods.

In [ ]:
class DBNClassifier(DBN):
    def __init__(self, layer_sizes, num_classes, learning_rate=0.1):
        """Initialize a DBN for classification."""
        # Initialize the base DBN
        super().__init__(layer_sizes, learning_rate)
        
        # Add a classification layer
        self.num_classes = num_classes
        self.output_weights = np.random.normal(0, 0.1, (layer_sizes[-1], num_classes))
        self.output_bias = np.zeros(num_classes)
        
    def softmax(self, x):
        """Compute softmax values for each set of scores in x."""
        e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return e_x / np.sum(e_x, axis=1, keepdims=True)
    
    def classify(self, data):
        """Forward pass through the network for classification."""
        # Pass through DBN layers
        h = data
        for rbm in self.rbm_layers:
            h, _ = rbm.sample_hidden(h)
        
        # Classification layer
        logits = np.dot(h, self.output_weights) + self.output_bias
        probs = self.softmax(logits)
        return probs
    
    def fine_tune(self, X_train, y_train, epochs=100, batch_size=32, learning_rate=0.01):
        """Fine-tune the DBN for classification."""
        # Convert labels to one-hot encoding
        y_one_hot = np.zeros((len(y_train), self.num_classes))
        for i, label in enumerate(y_train):
            y_one_hot[i, int(label)] = 1
        
        num_samples = X_train.shape[0]
        num_batches = num_samples // batch_size
        
        for epoch in range(epochs):
            # Shuffle data
            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            X_shuffled = X_train[indices]
            y_shuffled = y_one_hot[indices]
            
            epoch_loss = 0
            for batch in range(num_batches):
                start = batch * batch_size
                end = start + batch_size
                
                X_batch = X_shuffled[start:end]
                y_batch = y_shuffled[start:end]
                
                # Forward pass
                h = X_batch.copy()
                activations = [h]
                
                for rbm in self.rbm_layers:
                    h, _ = rbm.sample_hidden(h)
                    activations.append(h)
                
                # Output layer
                logits = np.dot(h, self.output_weights) + self.output_bias
                probs = self.softmax(logits)
                
                # Compute loss (cross entropy)
                batch_loss = -np.sum(y_batch * np.log(probs + 1e-10)) / batch_size
                epoch_loss += batch_loss
                
                # Backward pass (simplified backpropagation)
                # Output layer gradients
                delta = probs - y_batch
                output_grad = np.dot(activations[-1].T, delta) / batch_size
                output_bias_grad = np.mean(delta, axis=0)
                
                # Update output layer
                self.output_weights -= learning_rate * output_grad
                self.output_bias -= learning_rate * output_bias_grad
                
                # For simplicity, we're only updating the output layer here
                # In a full implementation, we would backpropagate through all layers
            
            avg_loss = epoch_loss / num_batches
            if epoch % 10 == 0 or epoch == epochs - 1:
                # Evaluate accuracy
                predictions = np.argmax(self.classify(X_train), axis=1)
                accuracy = np.mean(predictions == y_train)
                print(f"Epoch {epoch}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
    
    def evaluate(self, X_test, y_test):
        """Evaluate the classifier on test data."""
        predictions = np.argmax(self.classify(X_test), axis=1)
        accuracy = np.mean(predictions == y_test)
        return accuracy

## Training a DBN Classifier on MNIST

In [ ]:
# Create a subset for faster demonstration
n_train = 5000
X_train = X[:n_train]
y_train = np.array([int(label) for label in y[:n_train]])

# Create DBN classifier
input_size = X_train.shape[1]  # 784 for MNIST, 64 for digits
dbn_classifier = DBNClassifier(
    layer_sizes=[input_size, 250, 100], 
    num_classes=10, 
    learning_rate=0.01
)

# Pre-train the DBN
print("Pre-training DBN layers...")
dbn_classifier.pretrain(X_train, epochs=3, batch_size=10, k=1)

# Fine-tune for classification
print("\nFine-tuning for classification...")
dbn_classifier.fine_tune(X_train, y_train, epochs=50, batch_size=32, learning_rate=0.01)

# Evaluate on test set (we'll use a subset for speed)
n_test = 1000
X_test = X[n_train:n_train+n_test]
y_test = np.array([int(label) for label in y[n_train:n_train+n_test]])

accuracy = dbn_classifier.evaluate(X_test, y_test)
print(f"\nTest accuracy: {accuracy:.4f}")